# Basic Search

The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---
- **<a href="../../QuickStart/usage/#q">Q.()</a>** builds a query that can be used by `run()` or `count()`
- **<a href="../../QuickStart/usage/#qrun">Q.run()</a>** returns data for the specified search 
- **<a href="../../QuickStart/usage/#qcount">Q.count()</a>** returns summary information (counts) data that fit the specified search
- **<a href="../../QuickStart/usage/#columns">columns()</a>** returns entity field names
- **<a href="../../QuickStart/usage/#unique_terms">unique_terms()</a>** returns entity field contents

---
                                                                    
Before we do any work, we need to import these functions from cdapython. We're also importing functions from several other packages to make viewing and manipulating tables easier. The `opt.` settings are pre-configuring how itables should display our tables, with scrolling and paging enabled.
Finally, we're telling cdapython to report it's version so we can be sure we're using the one we mean to:

In [1]:
from cdapython import Q, columns, unique_terms, query
import numpy as np
import pandas as pd
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)
import itables.options as opt
opt.maxBytes=0
opt.scrollX="200px"
opt.scrollCollapse=True
opt.paging=True
opt.maxColumns=0
print(Q.get_version())

<IPython.core.display.Javascript object>

2022.10.24

In [2]:
# Delete me
Q.set_default_project_dataset("gdc-bq-sample.dev")
Q.set_table_version("all_Subjects_v3_1_final")
Q.set_host_url("http://35.192.60.10:8080/")
Q.get_host_url()
Q.get_default_project_dataset()

'gdc-bq-sample.dev'

<div class="cdanote" style="background-color:#b3e5d5;color:black;padding:20px;">
    
CDA data comes from three sources:
<ul>
<li><b>The <a href="https://proteomic.datacommons.cancer.gov/pdc/"> Proteomic Data Commons</a> (PDC)</b></li>
<li><b>The <a href="https://gdc.cancer.gov/">Genomic Data Commons</a> (GDC)</b></li>
<li><b>The <a href="https://datacommons.cancer.gov/repository/imaging-data-commons">Imaging Data Commons</a> (IDC)</b></li>
</ul> 
    
The CDA makes this data searchable in four main endpoints:

<ul>
<li><b>subject:</b> A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</li>
<li><b>researchsubject:</b> A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. A subject who participates in 3 studies will have 3 researchsubject IDs.</li>
<li><b>specimen:</b> Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</li>
<li><b>mutation:</b> Molecular data about specific mutations, currently limited to the TCGA-READ project from GDC.</li>
</ul>
one endpoint that can be added to the subject, researchsubject, or specimen to get the relevant files:    
<ul>
<li><b>file:</b> A unit of data about subjects, researchsubjects, specimens, or their associated information.</li>
</ul>
and two endpoints that offer deeper information about data in the researchsubject endpoint:
<ul>
<li><b>diagnosis:</b> A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</li>
<li><b>treatment:</b> Represent medication administration or other treatment types.</li>
</ul>
</div>


If you are looking to build a cohort of distinct individuals who meet some criteria, search by `subject`. If you want to build a cohort, but are particularly interested in studies rather than the participates per se, search by `researchsubject`. If you are looking for biosamples that can be ordered or a specific format of information (for e.g. histological slides) start with `specimen`. If you are primarily looking for files you can reuse for your own analysis, add `.file` to your call.

In the subject, researchsubject, or specimen tables, all of the rows will have one or more files associated with them that can be directly found by chaining, as in `specimen.files`. Diagnosis and treatment do not have files directly associated with them, so a query statemet of `diagnosis.files` or `treatment.file` will not work. The mutation table does have files associated with it, but currently they cannot be accessed with `mutation.file`. Look for this feature in a later release.

Any metadata field can be searched from any endpoint, the only difference between search types is what type of data is returned by default. This means that you can think of the CDA as a really, really enormous spreadsheet full of data. To search this enormous spreadsheet, you'd want select columns, and then filter rows.



## Basic search with endpoints

Let's try a broad search of the CDA to see what information exists about cancers that were first diagnosed in the brain. To run this simple search, we would first construct a query in `Q` where the syntax is `Q('<name_of_column> = "<unique_term>"'` and save it to a variable `myquery`:

In [3]:
myquery = Q('primary_diagnosis_site = "brain"')


<div class="cdawarn" style="background-color:#f9cfbf;color:black;padding:20px;">
    
<h3>Where did those terms come from?</h3>
    
If you aren't sure how we knew what terms to put in our search, please refer back to the <a href="../SearchTerms">What search terms are available?</a> notebook. 
</div>

### subject
Now we can use that query to search any of information types. Let's start by looking at what subjects meet our criteria. To do that, we will send our query to the subject endpoint, then ask for it to run:

In [4]:
subjectresults = myquery.subject.run()

Getting results from database

Total execution time: 0
                        min 4.349 sec 4349 ms

We saved the output in a variable `subjectresults`, so we don't get much visible output. To see what our results are, we need to look into the variable. The simplest way is to call `subjectresults` directly:

In [5]:
subjectresults


            
            Offset: 0
            Count: 100
            Total Row Count: 3015
            More pages: True
            

This tells us four parameters that describe our results:

---

- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows are in the full results table
- **More pages:** This is always a True or False. False means that our current page has all the available results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---

In any results variable, you can also use `print()` to display a 'pretty print' version of the results. This color codes the display in a way that can make it easier to find the information you care about:


In [6]:
print(subjectresults)

Offset: 0
            Count: 100
            Total Row Count: 3015
            More pages: True

We have 2384 subject matches, let's look at the actual table. The easiest way to do this is by using the python function `.to_dataframe()` on our `subjectresults` variable:

In [7]:
subjectresults.to_dataframe()

subject_id                                 subject_identifier  \
0               C177366            [{'system': 'PDC', 'value': 'C177366'}]   
1             C3N-03003          [{'system': 'IDC', 'value': 'C3N-03003'}]   
2                C48216             [{'system': 'PDC', 'value': 'C48216'}]   
3               C794457            [{'system': 'PDC', 'value': 'C794457'}]   
4                C82041             [{'system': 'PDC', 'value': 'C82041'}]   
..                  ...                                                ...   
95              C881049            [{'system': 'PDC', 'value': 'C881049'}]   
96  GENIE-GRCC-159a16b7  [{'system': 'GDC', 'value': 'GENIE-GRCC-159a16...   
97  GENIE-MSK-P-0005515  [{'system': 'GDC', 'value': 'GENIE-MSK-P-00055...   
98  QIN-BRAIN-DSC-01-33  [{'system': 'IDC', 'value': 'QIN-BRAIN-DSC-01-...   
99        QIN-GBM-TR-13      [{'system': 'IDC', 'value': 'QIN-GBM-TR-13'}]   

         species     sex                              race  \
0   homo sapiens    male  american indian or alaska native   
1   homo sapiens    None                              None   
2   homo sapiens  female                             white   
3   homo sapiens    male                             white   
4   homo sapiens    male                      not reported   
..           ...     ...                               ...   
95  homo sapiens    male                             white   
96  homo sapiens  female            not allowed to collect   
97  homo sapiens    male                             white   
98  homo sapiens    None                              None   
99  homo sapiens    None                              None   

                 ethnicity  days_to_birth  \
0   not hispanic or latino            NaN   
1                     None            NaN   
2   not hispanic or latino            NaN   
3   not hispanic or latino            NaN   
4   not hispanic or latino            NaN   
..                     ...            ...   
95  not hispanic or latino            NaN   
96  not allowed to collect            NaN   
97  not hispanic or latino            NaN   
98                    None            NaN   
99                    None            NaN   

                           subject_associated_project  vital_status  \
0   [Proteogenomic Analysis of Pediatric Brain Can...          Dead   
1                                         [cptac_gbm]          None   
2   [Proteogenomic Analysis of Pediatric Brain Can...         Alive   
3   [Proteogenomic Analysis of Pediatric Brain Can...         Alive   
4   [Proteogenomic Analysis of Pediatric Brain Can...         Alive   
..                                                ...           ...   
95  [Proteogenomic Analysis of Pediatric Brain Can...         Alive   
96                                       [GENIE-GRCC]  Not Reported   
97                                        [GENIE-MSK]  Not Reported   
98                                [qin_brain_dsc_mri]          None   
99                       [qin_gbm_treatment_response]          None   

    days_to_death cause_of_death  
0             NaN   Not Reported  
1             NaN           None  
2             NaN   Not Reported  
3             NaN   Not Reported  
4             NaN   Not Reported  
..            ...            ...  
95            NaN   Not Reported  
96            NaN           None  
97            NaN           None  
98            NaN           None  
99            NaN           None  

[100 rows x 11 columns]

By default `to_dataframe()` shows us the first and last five rows for the first page of our results, so we can easily preview our data.

Since we queried the Subject endpoint, our default results tell us Subject level information, that is, information about unique individuals: their sex, race, age, species, etc. The `id` column tells us the unique identifier for each individual. The identifier column has nested information about what study or studies a Subject participated in, and will list all of their researchsubject identifiers. 

The `to_dataframe()` function converts the results to a pandas dataframe. So if we save the dataframe to a variable, we can use any [pandas dataframe functions](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) to explore it. For example, lets see whether any of the Subjects in our first 100 results are black or african american. First we'll save the results to a dataframe, then subset that dataframe to only show rows where the word "black" appears in the "race" column. "NAs" which are shown as "None" in these tables, so for our filter to work, we'll need to specifically tell it to ignore NAs:


In [8]:
subjectdata = subjectresults.to_dataframe()
subjectdata[subjectdata['race'].str.contains("black", na=False)]

subject_id                                 subject_identifier  \
14  TCGA-06-A5U0  [{'system': 'GDC', 'value': 'TCGA-06-A5U0'}, {...   
41  TCGA-06-0876  [{'system': 'GDC', 'value': 'TCGA-06-0876'}, {...   
71  TCGA-06-0649  [{'system': 'GDC', 'value': 'TCGA-06-0649'}, {...   
73  TCGA-4W-AA9S  [{'system': 'GDC', 'value': 'TCGA-4W-AA9S'}, {...   

         species     sex                       race               ethnicity  \
14  homo sapiens  female  black or african american  not hispanic or latino   
41  homo sapiens  female  black or african american  not hispanic or latino   
71  homo sapiens  female  black or african american  not hispanic or latino   
73  homo sapiens    male  black or african american  not hispanic or latino   

    days_to_birth subject_associated_project vital_status  days_to_death  \
14            NaN       [TCGA-GBM, tcga_gbm]        Alive            NaN   
41            NaN       [TCGA-GBM, tcga_gbm]        Alive            NaN   
71            NaN       [TCGA-GBM, tcga_gbm]         Dead            NaN   
73            NaN       [TCGA-GBM, tcga_gbm]        Alive            NaN   

   cause_of_death  
14           None  
41           None  
71           None  
73           None

There are subjects in our first hundred results that meet the criteria. If we just want to be sure that the data contains some value, this might be good enough. But often we want to search the entire set of results and not just the first page. 

We'll cover how to work with large results dataframes in the <a href="../Pagination">Pagination</a> notebook. Or, learn how to get summary information from search results in the <a href="../DataSummaries">Data Summaries</a> notebook.

---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Subject Field Definitions</h3>

<i>A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</i>

    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system."</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>species:</b> The taxonomic group (e.g. species) of the subject.</li>
<li><b>sex:</b> The biologic character or quality that distinguishes male and female from one another as expressed by analysis of the person's gonadal, morphologic (internal and external), chromosomal, and hormonal characteristics.</li>
<li><b>race:</b> An arbitrary classification of a taxonomic group that is a division of a species. It usually arises as a consequence of geographical isolation within a species and is characterized by shared heredity, physical attributes and behavior, and in the case of humans, by common history, nationality, or geographic distribution. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>ethnicity:</b> An individual's self-described social and cultural grouping, specifically whether an individual describes themselves as Hispanic or Latino. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>days_to_birth:</b> Number of days between the date used for index and the date from a person's date of birth represented as a calculated negative number of days.</li>
<li><b>subject_associated_project:</b> The list of Projects associated with the Subject.</li>
<li><b>vital_status:</b> Coded value indicating the state or condition of being living or deceased; also includes the case where the vital status is unknown.</li>
<li><b>days_to_death:</b> Number of days between the date used for index and the date from a person's date of death represented as a calculated number of days.</li>
<li><b>cause_of_death:</b> Coded value indicating the circumstance or condition that results in the death of the subject.</li>
</ul>  

</div>
    
---

### researchsubject

If we're interested in what researchsubjects meet our criteria, we can run our query against the researchsubject endpoint:

In [9]:
researchsubjectresults = myquery.researchsubject.run()
print(researchsubjectresults)

Getting results from database

Total execution time: 0
                        min 7.779 sec 7779 ms

Offset: 0
            Count: 100
            Total Row Count: 4347
            More pages: True

Now we see that our 2384 subjects have 3716 researchsubjects between them, that means that some, but not all, of our subjects were participants in more than one study. Let's peek at the data:

In [10]:
researchsubjectresults.to_dataframe()

researchsubject_id  \
0   RIDER Neuro MRI-1205970977__rider_neuro_mri   
1             PGBM-019__brain_tumor_progression   
2             PGBM-002__brain_tumor_progression   
3   RIDER Neuro MRI-1596825834__rider_neuro_mri   
4          4dd86ebd-ef16-4b2b-9ea0-5d1d7afef257   
..                                          ...   
95                                  W19__ivygap   
96                                   W2__ivygap   
97                                   W3__ivygap   
98                                  W53__ivygap   
99                                  W16__ivygap   

                           researchsubject_identifier  \
0   [{'system': 'IDC', 'value': 'RIDER Neuro MRI-1...   
1   [{'system': 'IDC', 'value': 'PGBM-019__brain_t...   
2   [{'system': 'IDC', 'value': 'PGBM-002__brain_t...   
3   [{'system': 'IDC', 'value': 'RIDER Neuro MRI-1...   
4   [{'system': 'GDC', 'value': '4dd86ebd-ef16-4b2...   
..                                                ...   
95        [{'system': 'IDC', 'value': 'W19__ivygap'}]   
96         [{'system': 'IDC', 'value': 'W2__ivygap'}]   
97         [{'system': 'IDC', 'value': 'W3__ivygap'}]   
98        [{'system': 'IDC', 'value': 'W53__ivygap'}]   
99        [{'system': 'IDC', 'value': 'W16__ivygap'}]   

   member_of_research_project primary_diagnosis_condition  \
0             rider_neuro_mri                        None   
1     brain_tumor_progression                        None   
2     brain_tumor_progression                        None   
3             rider_neuro_mri                        None   
4                   TCGA-DLBC     Mature B-Cell Lymphomas   
..                        ...                         ...   
95                     ivygap                        None   
96                     ivygap                        None   
97                     ivygap                        None   
98                     ivygap                        None   
99                     ivygap                        None   

   primary_diagnosis_site                  subject_id  
0                   Brain  RIDER Neuro MRI-1205970977  
1                   Brain                    PGBM-019  
2                   Brain                    PGBM-002  
3                   Brain  RIDER Neuro MRI-1596825834  
4                   Brain                TCGA-RQ-AAAT  
..                    ...                         ...  
95                  Brain                         W19  
96                  Brain                          W2  
97                  Brain                          W3  
98                  Brain                         W53  
99                  Brain                         W16  

[100 rows x 6 columns]

Each row from the researchsubject endpoint results tells us about a subject in a given study. Using this endpoint we can find out information like what studies fit our search criteria, and also get data that we can filter to have only subjects from multiple studies, or only subjects from single studies.

Any given subject will have one row per study they participated in. The subject_id in the last column of this view is the same as the `id` in the first column of the Subjects endpoint results. You can use this to combine information across endpoints, which is covered near the end of the <a href="../BuildingACohort/#merging-results-across-endpoints">Cohort Building workflow</a> notebook.

---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>ResearchSubject Field Definitions</h3>

<i>A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. A subject who participates in 3 studies will have 3 researchsubject IDs</i>
    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. For CDA, this is case_id.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>member_of_research_project:</b> A reference to the Project(s) of which this ResearchSubject is a member.</li>
<li><b>primary_diagnosis_condition:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>primary_diagnosis_site:</b> The text term used to describe the primary site of disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This categorization groups cases into general categories. This attribute represents the primary site of disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>subject_id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. Can be joined to the `id` field from subject results</li>
</ul>  

</div>
    
---

### diagnosis

The diagnosis endpoint is an extension of the researchsubject endpoint, and returns information about researchsubjects that have a diagnosis that meets our search criteria:

In [11]:
diagnosisresults = myquery.diagnosis.run()
diagnosisresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 4.519 sec 4519 ms

diagnosis_id  \
0   021e5e64-6bb8-5d40-b61d-0eb41612fdf6   
1   15d04341-d00e-5a4d-a4c5-3688adca5bba   
2   17ad0147-433a-5d06-be6c-847ee8ed59d7   
3   2704dc4b-32e9-5043-8d37-8106fc8dffc1   
4   3b8fc610-2848-11ec-b712-0a4e2186f121   
..                                   ...   
95  986cc414-2545-495c-bad3-95df6c5bf505   
96  9b60596a-d804-4243-a24b-e94a8bb94189   
97  ae659079-fc5b-50a0-af51-1a52f60e9de4   
98  b07e96c3-742f-5f5a-9b64-558ee2f14ec4   
99  c97e5bdd-8495-5426-bcbf-f8780e4f39cf   

                                 diagnosis_identifier  \
0   [{'system': 'GDC', 'value': '021e5e64-6bb8-5d4...   
1   [{'system': 'GDC', 'value': '15d04341-d00e-5a4...   
2   [{'system': 'GDC', 'value': '17ad0147-433a-5d0...   
3   [{'system': 'GDC', 'value': '2704dc4b-32e9-504...   
4   [{'system': 'PDC', 'value': '3b8fc610-2848-11e...   
..                                                ...   
95  [{'system': 'GDC', 'value': '986cc414-2545-495...   
96  [{'system': 'GDC', 'value': '9b60596a-d804-424...   
97  [{'system': 'GDC', 'value': 'ae659079-fc5b-50a...   
98  [{'system': 'GDC', 'value': 'b07e96c3-742f-5f5...   
99  [{'system': 'GDC', 'value': 'c97e5bdd-8495-542...   

          primary_diagnosis  age_at_diagnosis morphology         stage  \
0   Astrocytoma, anaplastic           18977.0     9401/3          None   
1    Oligodendroglioma, NOS           11595.0     9450/3          None   
2              Glioblastoma           24319.0     9440/3          None   
3              Glioblastoma           31267.0     9440/3          None   
4              Glioblastoma           17038.0     9440/3  Not Reported   
..                      ...               ...        ...           ...   
95         Teratoma, benign               NaN     9080/0          None   
96             Glioblastoma           23267.0     9440/3          None   
97             Glioblastoma           23567.0     9440/3          None   
98             Mixed glioma           10790.0     9382/3          None   
99             Glioblastoma           21748.0     9440/3          None   

           grade method_of_diagnosis         subject_id  \
0   not reported                None       TCGA-CS-6665   
1   not reported                None       TCGA-DU-6401   
2   not reported                None       TCGA-76-4934   
3   not reported                None       TCGA-76-4928   
4   Not Reported                              C3L-01049   
..           ...                 ...                ...   
95  Not Reported                None  GENIE-DFCI-008091   
96  Not Reported                None          C3L-04084   
97  not reported                None       TCGA-81-5910   
98  not reported                None       TCGA-HT-7681   
99  not reported                None       TCGA-08-0359   

                      researchsubject_id  
0   c16d9f69-26f9-4ec7-b2d5-ff9f7dacfe0f  
1   97cb8701-7d63-4142-b404-006fbb3ecb0b  
2   d1fcd53b-6c55-42b0-9b24-492f8286f5bc  
3   dfa3ef71-7006-4c6d-81c4-e0c0c7b75c26  
4   104c11d9-2139-11ea-aee1-0e1aae319e49  
..                                   ...  
95  863f6e34-ebce-4133-87e0-108c5dc58221  
96  0345c668-0597-49a9-80b6-17cc9cc9cae7  
97  a63ce9ac-24a4-484e-8bcc-717cd7aa92df  
98  fbe32b94-b3ba-4a56-aeba-4cffe8de2da4  
99  0aa5b68f-2b99-44d2-b17f-f4eb17fe891e  

[100 rows x 10 columns]


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Diagnosis Field Definitions</h3>

<i>A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>primary_diagnosis:</b> The diagnosis instance that qualified a subject for inclusion on a ResearchProject.</li>
  <li><b>age_at_diagnosis:</b> The age in days of the individual at the time of diagnosis.</li>
  <li><b>morphology:</b> Code that represents the histology of the disease using the third edition of the <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a>, published in 2000, used principally in tumor and cancer registries for coding the site (topography) and the histology (morphology) of neoplasms.</li>
  <li><b>stage:</b> The extent of a cancer in the body. Staging is usually based on the size of the tumor, whether lymph nodes contain cancer, and whether the cancer has spread from the original site to other parts of the body. Different diseases may use different staging criteria, please refer to the originating data source to see what staging system is reported</li>
  <li><b>grade:</b> The degree of abnormality of cancer cells, a measure of differentiation, the extent to which cancer cells are similar in appearance and function to healthy cells of the same tissue type. The degree of differentiation often relates to the clinical behavior of the particular tumor. Based on the microscopic findings, tumor grade is commonly described by one of four degrees of severity. Histopathologic grade of a tumor may be used to plan treatment and estimate the future course, outcome, and overall prognosis of disease. Certain types of cancers, such as soft tissue sarcoma, primary brain tumors, lymphomas, and breast have special grading systems.</li>
  <li><b>method_of_diagnosis:</b> The method used to confirm the patients malignant diagnosis.</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---




### treatment

The treatment endpoint is an extension of diagnosis and returns information about treatments undertaken on researchsubjects that have a given diagnosis that meets our search criteria:

In [12]:
treatmentresults = myquery.treatment.run()
treatmentresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 4.61 sec 4610 ms

treatment_id  \
0   07c67ae3-8ece-5da4-be0f-e25a60f0f283   
1   0df1d067-698c-54e8-8d31-3fce30bc50a7   
2   1815e689-66a8-5eb2-932a-ff3c7fc9a083   
3   22952a1a-208e-44f2-8bc6-93c3049a0ca8   
4   273281ca-ba99-519d-bfa2-b94efac4a2c2   
..                                   ...   
95  65181048-74e8-5421-8572-c3af40cc413d   
96  7b93fd57-4573-410d-998a-898648696489   
97  81d84a22-1038-5707-8f3f-451fb6461527   
98  88f67a62-7030-5448-be4e-cde8a420b2b1   
99  89839070-3421-503b-87c6-81d12161d991   

                                 treatment_identifier treatment_type  \
0   [{'system': 'GDC', 'value': '07c67ae3-8ece-5da...           None   
1   [{'system': 'GDC', 'value': '0df1d067-698c-54e...           None   
2   [{'system': 'GDC', 'value': '1815e689-66a8-5eb...           None   
3   [{'system': 'GDC', 'value': '22952a1a-208e-44f...           None   
4   [{'system': 'GDC', 'value': '273281ca-ba99-519...           None   
..                                                ...            ...   
95  [{'system': 'GDC', 'value': '65181048-74e8-542...           None   
96  [{'system': 'GDC', 'value': '7b93fd57-4573-410...           None   
97  [{'system': 'GDC', 'value': '81d84a22-1038-570...           None   
98  [{'system': 'GDC', 'value': '88f67a62-7030-544...           None   
99  [{'system': 'GDC', 'value': '89839070-3421-503...           None   

   treatment_outcome days_to_treatment_start days_to_treatment_end  \
0               None                    None                  None   
1               None                    None                  None   
2               None                    None                  None   
3               None                    None                  None   
4               None                    None                  None   
..               ...                     ...                   ...   
95              None                    None                  None   
96              None                    None                  None   
97              None                    None                  None   
98              None                    None                  None   
99              None                    None                  None   

   therapeutic_agent treatment_anatomic_site treatment_effect  \
0               None                    None             None   
1               None                    None             None   
2               None                    None             None   
3               None                    None             None   
4               None                    None             None   
..               ...                     ...              ...   
95              None                    None             None   
96        Irinotecan                    None             None   
97              None                    None             None   
98              None                    None             None   
99              None                    None             None   

   treatment_end_reason number_of_cycles         subject_id  \
0                  None             None       TCGA-08-0518   
1                  None             None       TCGA-12-0827   
2                  None             None       TCGA-76-6656   
3                  None             None  HCM-BROD-0106-C71   
4                  None             None       TCGA-02-0325   
..                  ...              ...                ...   
95                 None             None       TCGA-06-6694   
96                 None             None            ER-ACXF   
97                 None             None       TCGA-02-0258   
98                 None             None       TCGA-06-1802   
99                 None             None       TCGA-RY-A83Y   

                      researchsubject_id                          diagnosis_id  
0   64fa7f09-724a-45ee-a802-09e4affe077e  85441794-e4dd-54a4-88f9-16e8c3a0a585  
1   b75845ed-a19a-4caa-b362-e62b1e538152  65ef1c26-0098-511e-b1ce-2e5df9d62f87  
2   770aa1ee


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Treatment Field Definitions</h3>

<i><i> Medication administration or other treatment types. A single research subject may have multiple treatments for a single diagnosis, and/or different diagnoses, and different treatments, across different studies</i></i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>treatment_type:</b> The treatment type including medication/therapeutics or other procedures.</li>
  <li><b>treatment_outcome:</b> The final outcome of the treatment.</li>
  <li><b>days_to_treatment_start:</b> The timepoint at which the treatment started.</li>
  <li><b>days_to_treatment_end:</b>The timepoint at which the treatment ended. </li>
  <li><b>therapeutic_agent:</b> One or more therapeutic agents as part of this treatment.</li>
  <li><b>treatment_anatomic_site:</b> The anatomical site that the treatment targets.</li>
  <li><b>treatment_effect:</b>The effect of a treatment on the diagnosis or tumor. </li>
  <li><b>treatment_end_reason:</b>The reason the treatment ended. </li>
  <li><b>number_of_cycles:</b>The number of treatment cycles the subject received. </li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
  <li><b>researchsubject_diagnosis_id:</b> An identifier for the diagnosis. Can be joined to the `id` field from diagnosis results</li>
</ul>  

</div>
    
---




### specimen

We can use this same query to see what specimens are available for brain tissue at the CDA:

In [13]:
specimenresults =  myquery.specimen.run()
specimenresults

Getting results from database

Total execution time: 0
                        min 7.896 sec 7896 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 39220
            More pages: True
            

Nearly 40,000 specimens meet our search criteria! We would typically expect this number to be much larger than our number of subjects or researchsubjects. First because studies will often take more than one sample per subject, and second because any given specimen might be aliquoted out to be used in multiple tests. Since we didn't specify any further filters, our results will return all of these as separate speciments. Let's look at a few:

In [14]:
specimenresults.to_dataframe()

specimen_id  \
0   0024bf91-4ba7-4e0d-9e23-cb37188324c7   
1   01b98b0e-bbfc-4e0f-934a-3a6ac9486342   
2   01ec04df-91c8-4c46-a71d-fe1abe06a221   
3   0242c021-1602-420e-a245-8ee1052fdcef   
4   031453eb-67ac-416e-8350-2e35f55c7773   
..                                   ...   
95  3abeff8e-5bc2-4cb9-aa06-55bd17d41f82   
96  3b60125e-2eed-4a75-9811-b7749dbe7977   
97  3b9c4291-4b37-413e-877e-0b557e960c64   
98  3bc3a6ac-de5e-4a7b-8fea-a9cf6f4bfd18   
99  3c83ded3-3515-4071-8788-6e8bf2c2a173   

                                  specimen_identifier  \
0   [{'system': 'GDC', 'value': '0024bf91-4ba7-4e0...   
1   [{'system': 'GDC', 'value': '01b98b0e-bbfc-4e0...   
2   [{'system': 'GDC', 'value': '01ec04df-91c8-4c4...   
3   [{'system': 'GDC', 'value': '0242c021-1602-420...   
4   [{'system': 'GDC', 'value': '031453eb-67ac-416...   
..                                                ...   
95  [{'system': 'GDC', 'value': '3abeff8e-5bc2-4cb...   
96  [{'system': 'GDC', 'value': '3b60125e-2eed-4a7...   
97  [{'system': 'GDC', 'value': '3b9c4291-4b37-413...   
98  [{'system': 'GDC', 'value': '3bc3a6ac-de5e-4a7...   
99  [{'system': 'GDC', 'value': '3c83ded3-3515-407...   

   specimen_associated_project  days_to_collection primary_disease_type  \
0                     TCGA-LGG                78.0              Gliomas   
1                     TCGA-LGG                 NaN              Gliomas   
2                     TCGA-LGG               219.0              Gliomas   
3                     TCGA-LGG                 0.0              Gliomas   
4                     TCGA-GBM                 NaN              Gliomas   
..                         ...                 ...                  ...   
95                    TCGA-GBM                 NaN              Gliomas   
96                    TCGA-LGG                 NaN              Gliomas   
97                    TCGA-GBM                 NaN              Gliomas   
98                    TCGA-GBM                 NaN              Gliomas   
99                    TCGA-GBM                 NaN              Gliomas   

   anatomical_site  source_material_type specimen_type  \
0             None  Blood Derived Normal       portion   
1             None         Primary Tumor       portion   
2             None         Primary Tumor       aliquot   
3             None         Primary Tumor         slide   
4             None         Primary Tumor       analyte   
..             ...                   ...           ...   
95            None         Primary Tumor       aliquot   
96            None  Blood Derived Normal       aliquot   
97            None  Blood Derived Normal       portion   
98            None  Blood Derived Normal       portion   
99            None         Primary Tumor        sample   

                   derived_from_specimen derived_from_subject    subject_id  \
0   3407f4e4-02ef-49aa-bea0-f7c496ad7371         TCGA-S9-A7QW  TCGA-S9-A7QW   
1   ae23f336-e901-4e45-b139-5e105060d3af         TCGA-HT-7609  TCGA-HT-7609   
2   183243f7-1294-490c-a5b8-af6bd8740e01         TCGA-HW-A5KL  TCGA-HW-A5KL   
3   b44d1202-4b05-545c-8f00-fdad812c28ef         TCGA-HT-7689  TCGA-HT-7689   
4   0efe9cfc-89f2-4096-8c3e-c300a8efa1ee         TCGA-02-2486  TCGA-02-2486   
..                                   ...                  ...           ...   
95  0ed7f9eb-ac18-45a7-8b0c-f4912d2d1e2d         TCGA-32-1970  TCGA-32-1970   
96  05b422cc-7cc1-4691-a337-ed5f751b7c10         TCGA-DU-8167  TCGA-DU-8167   
97  aa3459f6-b50c-4205-8e73-d07fc50ec0ac         TCGA-27-2526  TCGA-27-2526   
98  ea567da7-cbca-4657-9762-918c513c768c         TCGA-19-1791  TCGA-19-1791   
99                      initial specimen         TCGA-19-4065  TCGA-19-4065   

                      researchsubject_id  
0   765588aa-075f-477f-9c18-099d018d3a1b  
1   bfbc0d1e-2a0b-4c60-8a37-91e3ce4b79ee  
2   0639b753-4099-4ca4-adb9-763069a614ef  
3   d58b535a-6c95-43bb-ad88-0600e6447537  
4   79a68b32-29a3-47bf-9b2b-643ec7323d73  
..        


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Specimen Field Definitions</h3>

<i><i>Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</i>
 A given specimen will have only a single subject ID and a single researchsubject ID</i>
    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>associated_project:</b> The Project associated with the specimen.</li>
  <li><b>days_to_collection:</b> The number of days from the index date to either the date a sample was collected for a specific study or project, or the date a patient underwent a procedure (e.g. surgical resection) yielding a sample that was eventually used for research.</li>
  <li><b>primary_disease_type:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
  <li><b>anatomical_site:</b> Per GDC Dictionary, the text term that represents the name of the primary disease site of the submitted tumor sample; recommend dropping tumor; biospecimen_anatomic_site.</li>
  <li><b>source_material_type:</b> The general kind of material from which the specimen was derived, indicating the physical nature of the source material.</li>
  <li><b>specimen_type:</b> The high-level type of the specimen, based on its how it has been derived from the original extracted sample. One of: analyte, aliquot, portion, sample, or slide.</li>
  <li><b>derived_from_specimen:</b> A source/parent specimen from which this one was directly derived.</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---


### file

The file endpoint returns information about files that meet our search criteria, and that belong to subjects, researchsubjects, or specimens. You can think of three of these endpoints as being nested: 

`subject.file` is the broadest search. It will give you all files that have the parameters of your query
`researchsubject.file` will give you all the files that match your query, and that are attached to specific studies and/or specimens. It will not return subject level files.
`specimen.file` will return all the files directly attached to specimen, but not those attached to the researchsubject or subject those specimens came from.


In [15]:
myquery.subject.file.run()

Getting results from database

Total execution time: 0
                        min 22.472 sec 22472 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 4969686
            More pages: True
            

In [16]:
myquery.researchsubject.file.run()

Getting results from database

Total execution time: 0
                        min 24.53 sec 24530 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 4969666
            More pages: True
            

In [17]:
myquery.specimen.file.run()

Getting results from database

Total execution time: 0
                        min 10.649 sec 10649 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 771260
            More pages: True
            

As you might expect, searching file gives us a huge number of results. This is great if you are surveying what kind of data is available, but is less useful for getting a coherent cohort. 

A better way to get files for a specific cohort is to build a more complex query using <a href="../Operators">Operators</a>. 

Another useful way to look at high level information is to use our counts feature which returns summary information rather than the full search results. Check out the <a href="../DataSummaries">Data Summaries tutorial</a> to try it.



---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>File Field Definitions</h3>

<i>A file is an information-bearing electronic object that contains a physical embodiment of some information using a particular character encoding.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>label:</b> Short name or abbreviation for dataset. Maps to rdfs:label.</li>
  <li><b>data_catagory:</b> Broad categorization of the contents of the data file.</li>
  <li><b>data_type:</b> Specific content type of the data file.</li>
  <li><b>file_format:</b> Format of the data files.</li>
  <li><b>associated_project:</b> A reference to the Project(s) of which this ResearchSubject is a member. The associated_project may be embedded using the ref definition or may be a reference to the id for the Project - or a URI expressed as a string to an existing entity.</li>
  <li><b>drs_uri:</b> A string of characters used to identify a resource on the Data Repo Service(DRS). Can be used to retrieve this specific file from a server.</li>
  <li><b>byte_size:</b> Size of the file in bytes. Maps to dcat:byteSize.</li>
  <li><b>checksum:</b> The md5 value for the file. A digit representing the sum of the correct digits in a piece of stored or transmitted digital data, against which later comparisons can be made to detect errors in the data.</li>
  <li><b>data_modality:</b> Data modality describes the biological nature of the information gathered as the result of an activity, independent of the technology or methods used to produce the information. Always one of "Genomic", "Proteomic", or "Imaging".</li>
  <li><b>imaging_modality:</b> An imaging modality describes the imaging equipment and/or method used to acquire certain structural or functional information about the body. These include but are not limited to computed tomography (CT) and magnetic resonance imaging (MRI). Taken from the DICOM standard.</li>
  <li><b>dbgap_accession_number:</b> The dbgap accession number for the project.</li>
</ul>  

</div>
    
---

### mutation

The file endpoint returns molecular data about specific mutations that meet our search criteria. Search is currently limited to the TCGA-READ project from GDC:

In [18]:
myquery.mutation.run()

Getting results from database

Total execution time: 0
                        min 16.393 sec 16393 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 904
            More pages: True
            

In [19]:
mutationresults = myquery.mutation.run()
mutationresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 4.067 sec 4067 ms

project_short_name  case_barcode sample_barcode_tumor  \
0            TCGA-GBM  TCGA-06-5415     TCGA-06-5415-01A   
1            TCGA-GBM  TCGA-32-5222     TCGA-32-5222-01A   
2            TCGA-GBM  TCGA-41-5651     TCGA-41-5651-01A   
3            TCGA-LGG  TCGA-HT-8109     TCGA-HT-8109-01A   
4            TCGA-LGG  TCGA-HW-A5KL     TCGA-HW-A5KL-01A   
..                ...           ...                  ...   
95           TCGA-LGG  TCGA-HT-7677     TCGA-HT-7677-01A   
96           TCGA-GBM  TCGA-14-0787     TCGA-14-0787-01A   
97           TCGA-GBM  TCGA-14-1821     TCGA-14-1821-01A   
98           TCGA-LGG  TCGA-DU-7011     TCGA-DU-7011-01A   
99           TCGA-LGG  TCGA-DU-7290     TCGA-DU-7290-01A   

   sample_barcode_normal         aliquot_barcode_tumor  \
0       TCGA-06-5415-10A  TCGA-06-5415-01A-01D-1486-08   
1       TCGA-32-5222-10A  TCGA-32-5222-01A-01D-1486-08   
2       TCGA-41-5651-10A  TCGA-41-5651-01A-01D-1696-08   
3       TCGA-HT-8109-10A  TCGA-HT-8109-01A-11D-2395-08   
4       TCGA-HW-A5KL-10A  TCGA-HW-A5KL-01A-11D-A27K-08   
..                   ...                           ...   
95      TCGA-HT-7677-10A  TCGA-HT-7677-01A-11D-2253-08   
96      TCGA-14-0787-10A  TCGA-14-0787-01A-01D-1492-08   
97      TCGA-14-1821-10A  TCGA-14-1821-01A-01W-0643-08   
98      TCGA-DU-7011-10A  TCGA-DU-7011-01A-11D-2024-08   
99      TCGA-DU-7290-10A  TCGA-DU-7290-01A-11D-2024-08   

          aliquot_barcode_normal Hugo_Symbol  Entrez_Gene_Id Center  \
0   TCGA-06-5415-10A-01D-1486-08       RASA1            5921     BI   
1   TCGA-32-5222-10A-01D-1486-08       HECW1           23072     BI   
2   TCGA-41-5651-10A-01D-1696-08    CDC42EP3           10602     BI   
3   TCGA-HT-8109-10A-01D-2396-08       LOXL4           84171     BI   
4   TCGA-HW-A5KL-10A-01D-A27N-08       UBXN1           51035     BI   
..                           ...         ...             ...    ...   
95  TCGA-HT-7677-10A-01D-2253-08      ARID1B           57492     BI   
96  TCGA-14-0787-10A-01D-1492-08       NOP56           10528     BI   
97  TCGA-14-1821-10A-01W-0644-08        TP53            7157     BI   
98  TCGA-DU-7011-10A-01D-2024-08      SIRPB1           10326     BI   
99  TCGA-DU-7290-10A-01D-2024-08       MXRA5           25878     BI   

   NCBI_Build  ...                        tumor_bam_uuid  \
0      GRCh38  ...  b49524b5-c615-4893-895d-bc2a24688ffa   
1      GRCh38  ...  44e42588-b00e-4282-9f57-74c15b5aee1a   
2      GRCh38  ...  06526eec-5632-4c24-a035-274bd3118661   
3      GRCh38  ...  d25043dd-2196-496a-aeeb-5adadf1adb84   
4      GRCh38  ...  3ad19e57-5d90-4624-9320-9320d1535086   
..        ...  ...                                   ...   
95     GRCh38  ...  359da179-14d2-4e7c-901e-864f03cdad8d   
96     GRCh38  ...  211cf8f0-d56b-447e-9fd8-f2622b75e94b   
97     GRCh38  ...  dd7b9f39-ebd0-4873-9423-74e6a9eeeba6   
98     GRCh38  ...  265b7262-df74-4239-a5f6-77c326cf5f4e   
99     GRCh38  ...  cbf5d324-9fa0-4e81-a5a1-0a1d447a1f19   

                         normal_bam_uuid  \
0   0a26c1a1-5900-4dac-b5ee-bf7b94fbafb2   
1   41a523dd-b452-4c4c-b46b-6461cf478fcc   
2   950aea08-5e79-4aa6-b015-0f9cb6836211   
3   0e6b0fb2-46ff-4901-be54-926f4bed5f73   
4   a377e4b9-2de0-457c-87bc-1253f14a42ca   
..                                   ...   
95  f29abb60-b6a6-4742-a4e8-9656c501f4b8   
96  b2250dde-b989-4e6b-a075-ea96326bd3aa   
97  49b9ee4c-ab34-4ac9-8524-12cf6ca1b6cf   
98  e9c5af45-ba04-4ea4-ac71-7c48fc9205f0   
99  7803ef81-54b1-40f8-8093-19d608b2c8dd   

                                 case_id GDC_FILTER  \
0   713190ed-c6c1-4695-814b-85ca9b95e6a0       None   
1   e876dd29-68b0-4bf1-83d1-488c40068a35       None   
2   c184c3ca-7ad3-4202-b108-cb9fd5f5d947       None   
3   c7b2833a-9fe8-49f6-b888-2d27b0516f5c       None   
4   0639b753-4099-4ca4-adb9-763069a614ef       None   
..                                   ...        ...   
95  00b18ad6-ff70-41fe-8f4d-0763ac8f28b9       None   
96  a2338b30-f511-4163-af3b-1e4a40ff00e1  